In [4]:
import heapq
import time

# ===================== Cube Representation =====================
class RubiksCube:
    U, R, F, D, L, B = range(6)

    def __init__(self):
        colors = ['w', 'r', 'g', 'y', 'o', 'b']
        self.faces = [[c]*9 for c in colors]

    def apply_move(self, move):
        if move == "U": self.rotate_u()
        elif move == "U'": self.rotate_u(); self.rotate_u(); self.rotate_u()
        elif move == "U2": self.rotate_u(); self.rotate_u()
        elif move == "R": self.rotate_r()
        elif move == "R'": self.rotate_r(); self.rotate_r(); self.rotate_r()
        elif move == "R2": self.rotate_r(); self.rotate_r()
        elif move == "F": self.rotate_f()
        elif move == "F'": self.rotate_f(); self.rotate_f(); self.rotate_f()
        elif move == "F2": self.rotate_f(); self.rotate_f()

    def rotate_face_clockwise(self, f):
        t = self.faces[f][:]
        self.faces[f][0] = t[6]
        self.faces[f][1] = t[3]
        self.faces[f][2] = t[0]
        self.faces[f][3] = t[7]
        self.faces[f][4] = t[4]
        self.faces[f][5] = t[1]
        self.faces[f][6] = t[8]
        self.faces[f][7] = t[5]
        self.faces[f][8] = t[2]

    def rotate_u(self):
        self.rotate_face_clockwise(self.U)
        t = self.faces[self.F][0:3]
        self.faces[self.F][0:3] = self.faces[self.L][0:3]
        self.faces[self.L][0:3] = self.faces[self.B][0:3]
        self.faces[self.B][0:3] = self.faces[self.R][0:3]
        self.faces[self.R][0:3] = t

    def rotate_r(self):
        self.rotate_face_clockwise(self.R)
        t = [self.faces[self.U][2], self.faces[self.U][5], self.faces[self.U][8]]
        self.faces[self.U][2], self.faces[self.U][5], self.faces[self.U][8] = \
            self.faces[self.F][2], self.faces[self.F][5], self.faces[self.F][8]
        self.faces[self.F][2], self.faces[self.F][5], self.faces[self.F][8] = \
            self.faces[self.D][2], self.faces[self.D][5], self.faces[self.D][8]
        self.faces[self.D][2], self.faces[self.D][5], self.faces[self.D][8] = \
            self.faces[self.B][6], self.faces[self.B][3], self.faces[self.B][0]
        self.faces[self.B][6], self.faces[self.B][3], self.faces[self.B][0] = t

    def rotate_f(self):
        self.rotate_face_clockwise(self.F)
        t = [self.faces[self.U][6], self.faces[self.U][7], self.faces[self.U][8]]
        self.faces[self.U][6] = self.faces[self.L][8]
        self.faces[self.U][7] = self.faces[self.L][5]
        self.faces[self.U][8] = self.faces[self.L][2]
        self.faces[self.L][8] = self.faces[self.D][2]
        self.faces[self.L][5] = self.faces[self.D][1]
        self.faces[self.L][2] = self.faces[self.D][0]
        self.faces[self.D][0] = self.faces[self.R][6]
        self.faces[self.D][1] = self.faces[self.R][3]
        self.faces[self.D][2] = self.faces[self.R][0]
        self.faces[self.R][0] = t[2]
        self.faces[self.R][3] = t[1]
        self.faces[self.R][6] = t[0]

    def serialize(self):
        # Fast, hashable representation
        return tuple(tuple(face) for face in self.faces)

# ===================== Heuristic (Improved) =====================
def heuristic(cube):
    total = 0
    for f in range(6):
        center = cube.faces[f][4]
        total += sum(1 for i in range(9) if cube.faces[f][i] != center)
    return total

# ===================== A* Solver =====================
class State:
    def __init__(self, cube, moves, cost, heuristic_val):
        self.cube = cube
        self.moves = moves
        self.cost = cost
        self.heuristic = heuristic_val

    def __lt__(self, other):
        return (self.cost + self.heuristic) < (other.cost + other.heuristic)

all_moves = ["U", "U'", "U2", "R", "R'", "R2", "F", "F'", "F2"]

def get_valid_moves(prev_move):
    if not prev_move:
        return all_moves
    prev_face = prev_move[0]
    return [m for m in all_moves if m[0] != prev_face]

def solve_cube(start_cube):
    heap = []
    visited = set()
    heapq.heappush(heap, State(start_cube, [], 0, heuristic(start_cube)))

    while heap:
        state = heapq.heappop(heap)
        if state.heuristic == 0:
            return state.moves

        key = state.cube.serialize()
        if key in visited:
            continue
        visited.add(key)

        for move in get_valid_moves(state.moves[-1] if state.moves else None):
            next_cube = RubiksCube()
            next_cube.faces = [face[:] for face in state.cube.faces]
            next_cube.apply_move(move)

            new_moves = state.moves + [move]
            heapq.heappush(heap, State(next_cube, new_moves, state.cost + 1, heuristic(next_cube)))

    return []

# ===================== MAIN =====================
def main():
    cube = RubiksCube()
    print("=== Rubik's Cube Solver (A* Search) ===")
    print("Enter scramble moves separated by spaces (e.g., U R F U'):")
    print("Allowed moves: U, U', U2, R, R', R2, F, F', F2")
    line = input("> ").strip()
    scramble = line.split()

    for move in scramble:
        cube.apply_move(move)

    print("\nScrambled Cube Applied! Starting Solver...")

    start_time = time.time()
    solution = solve_cube(cube)
    end_time = time.time()

    print("\nSolution Moves:", ' '.join(solution))
    print("Number of Moves:", len(solution))
    print("Time Taken: {:.3f} ms".format((end_time - start_time) * 1000))

if __name__ == "__main__":
    main()


=== Rubik's Cube Solver (A* Search) ===
Enter scramble moves separated by spaces (e.g., U R F U'):
Allowed moves: U, U', U2, R, R', R2, F, F', F2
> U R' U R

Scrambled Cube Applied! Starting Solver...

Solution Moves: R' U' R U'
Number of Moves: 4
Time Taken: 9.703 ms
